# Setup

The main focus of this notebook is to analyze and mitigate potential biases in a dataset. The notebook uses various techniques and technologies, such as [AIF360](https://aif360.mybluemix.net), [Fairlearn](https://fairlearn.org) , and [scikit-learn](https://scikit-learn.org/stable/), to preprocess and analyze the data, and to train and evaluate machine learning models. The notebook also includes visualizations and statistics to help understand the distribution and correlations of the data, and to identify any potential biases.

**PLEASE NOTE**: The notebook must be configured with a dataset and some configuration variables.  
The ***protected attributes*** must be categorical and binary (0,1), but the original column must be mantained as a continuous variable.

## Colab
In order to run this notebook in Google Colab, you have to:
1. Upload the project folder to your Google Drive
2. Mount your Google Drive in the next code cell
3. Update the `path_to_project` variable, with the path to the project folder in your Google Drive (e.g. `path_to_project = '/content/drive/MyDrive/Colab/project'`), this can be seen from the file tab on the left of the Colab interface
4. Update the pip installation command  (e.g. `!pip install -r /content/drive/MyDrive/Colab/project/requirements.txt`)

In [ ]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  import sys
  path_to_project = '/content/drive/MyDrive/FairAlgorithm'  #UPDATE THIS LINE
  sys.path.append(path_to_project)
  !sudo apt install libcairo2-dev pkg-config python3-dev
  %pip install -r /content/drive/MyDrive/FairAlgorithm/source/requirements.txt  #UPDATE THIS LINE
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
import numpy as np
import pandas as pd
np.random.seed(1234)
from rich import print
from rich.columns import Columns
from rich.panel import Panel
from rich.align import Align
from source.utils.print_util import *
from source.utils.data_preprocessing import *
import matplotlib.pyplot as plt
from fairlearn.metrics import  MetricFrame, count, false_negative_rate, false_positive_rate, selection_rate, equalized_odds_difference, demographic_parity_difference
from fairlearn.preprocessing import CorrelationRemover
from fairlearn.adversarial import AdversarialFairnessClassifier
from fairlearn.reductions import ExponentiatedGradient, Moment
from fairlearn.postprocessing import ThresholdOptimizer
from aif360.datasets import BinaryLabelDataset, StructuredDataset, StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric, Metric
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing, LFR, OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
from aif360.algorithms.inprocessing import PrejudiceRemover, AdversarialDebiasing, ExponentiatedGradientReduction
from aif360.algorithms.postprocessing import RejectOptionClassification, CalibratedEqOddsPostprocessing, EqOddsPostprocessing
from aif360.algorithms import Transformer

from tqdm.notebook import tqdm
import pickle

pip install 'aif360[LawSchoolGPA]'


In [ ]:
from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict, train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, recall_score, accuracy_score, precision_score, confusion_matrix, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

## Configure the notebook


In the next code cell, set all the variables that are used throughout the notebook.  
The variables are used to configure the notebook, and to set the paths to the data files.

Modify the variables in the next code cell to configure the notebook

- `dataset_name`: The name of the dataset file.
- `dataset_path`: The path to the dataset file.
- `target`: The target feature to predict.
- `target_variable_labels`: The labels for the target feature.
- `sensible_attribute`: The sensible attribute to use for bias mitigation.

In [ ]:
#INPUT
dataset_name = "diabetes-women"
dataset_path = path_to_project + '/data/preprocessed/preprocessed-{}.csv'.format(dataset_name) if IN_COLAB else 'data/preprocessed/preprocessed-{}.csv'.format(dataset_name)

ignore_cols = ['Age']
target_variable = 'Outcome'
target_variable_labels= ['No Diabetes','Diabetes']
sensible_attribute = 'AgeCategory'
random_seed = 1234

In [ ]:
default_mappings = {
    'label_maps': [{1.0: 'Diabetic', 0.0: 'NonDiabetic'}],
    'protected_attribute_maps': [{1.0: 'Adult', 0.0: 'Young'}]
}

In [ ]:
n_estimators = 30
random_seed = 1234

models = {'Logistic Regression':LogisticRegression(max_iter=500),
          'Decision Tree':DecisionTreeClassifier(max_depth=None),
          'Bagging':BaggingClassifier(DecisionTreeClassifier(max_depth=3),n_estimators=n_estimators),
          'Random Forest':RandomForestClassifier(n_estimators=n_estimators),
          'Extremely Randomized Trees':ExtraTreesClassifier(n_estimators=n_estimators),
          'Ada Boost':AdaBoostClassifier(DecisionTreeClassifier(max_depth=3),n_estimators=n_estimators)}

In [ ]:
df = pd.read_csv(dataset_path)
df = df.drop(columns=ignore_cols)
feature_cols = df.columns

## Load the data
Data is loaded from the file specified in the `dataset_path` variable using the `pandas` library.  
[pandas](https://pandas.pydata.org) is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool,
built on top of the Python programming language.

In [ ]:
df.head(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Outcome,AgeCategory
0,6,148,72,35,0,33.6,0.627,1,1
1,1,85,66,29,0,26.6,0.351,0,1
2,8,183,64,0,0,23.3,0.672,1,1
3,1,89,66,23,94,28.1,0.167,0,0
4,0,137,40,35,168,43.1,2.288,1,1


## Mitigation configuration

Non sono convinta che questo sia il processo migliore per calcolare le predizioni su cui misurare la fairness.

Forse spezzerei il dataset in due parti: train e test (rendendo entrambi le parti rappresentative) e valuterei la parte di fairness solo sul test set

In [ ]:
def compute_predictions(df):
  n_estimators = 30
  random_seed = 1234

  models = {'Logistic Regression':LogisticRegression(max_iter=500),
            'Decision Tree':DecisionTreeClassifier(max_depth=None),
            'Bagging':BaggingClassifier(DecisionTreeClassifier(max_depth=3),n_estimators=n_estimators),
            'Random Forest':RandomForestClassifier(n_estimators=n_estimators),
            'Extremely Randomized Trees':ExtraTreesClassifier(n_estimators=n_estimators),
            'Ada Boost':AdaBoostClassifier(DecisionTreeClassifier(max_depth=3),n_estimators=n_estimators)}

  scores = {}
  predicted_values = {}
  for model_name in tqdm(models):
      X = df.drop(target_variable, axis=1)
      y = df[target_variable].values
      clf = models[model_name]
      cross_val_results = cross_val_predict(clf,X,y,cv=StratifiedKFold(n_splits=10,shuffle=True,random_state=random_seed))
      predicted_values[model_name] = cross_val_results

      score = cross_val_score(clf,X,y,cv=StratifiedKFold(n_splits=10,shuffle=True,random_state=random_seed))
      scores[model_name] = [round(np.average(score), 5), round(np.std(score), 5)]
  return predicted_values, scores

In [ ]:
def save_predictions_scores(predicted_values, scores, mitigation, dataset_name):
  save_path = path_to_project + '/data/predictions/predictions-{}-{}.p'.format(dataset_name, mitigation)
  with open(save_path, 'wb') as fp:
    pickle.dump(predicted_values, fp, protocol=pickle.HIGHEST_PROTOCOL)

  if scores != None:
    save_path = path_to_project + '/data/scores/scores-{}-{}.p'.format(dataset_name, mitigation)
    with open(save_path, 'wb') as fp:
        pickle.dump(scores, fp, protocol=pickle.HIGHEST_PROTOCOL)

# Fairlearn
[Fairlearn](https://fairlearn.org) is an open source toolkit that empowers developers of artificial intelligence (AI) systems to assess their system's fairness and mitigate any observed unfairness issues.

### Correlation Remover
Pre-processing: mitigated dataset changes

In [ ]:
def plot_heatmap(df, title, y):
    df["target"] = y
    df = df.rename(columns={"had_inpatient_days_True": "had_inpatient_days"})
    cols = list(df.columns)

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.imshow(round(df.corr(), 2), cmap="coolwarm")

    # Show all ticks and label them with the respective list entries
    ax.set_xticks(np.arange(len(cols)))
    ax.set_xticklabels(cols)
    ax.set_yticks(np.arange(len(cols)))
    ax.set_yticklabels(cols)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=15, ha="right", rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    for i in range(len(cols)):
        for j in range(len(cols)):
            ax.text(
                j,
                i,
                round(df.corr().to_numpy()[i, j], 2),
                ha="center",
                va="center",
            )

    ax.set_title(f"{title}")
    plt.show()

In [ ]:
X_raw = df.drop(target_variable, axis=1)
X_raw = pd.get_dummies(X_raw)
y = df[target_variable].values

cr = CorrelationRemover(sensitive_feature_ids=[sensible_attribute])
X_cr = cr.fit_transform(X_raw)

X_cr = pd.DataFrame(
    X_cr, columns = X_raw.drop(sensible_attribute, axis=1).columns
)

X_cr[sensible_attribute] = X_raw[sensible_attribute]
mit_fl_cr = X_cr.copy(deep=True)
mit_fl_cr[target_variable] = y

In [ ]:
# cr_alpha = CorrelationRemover(sensitive_feature_ids=["race_AfricanAmerican"], alpha=0.5)
# X_cr_alpha = cr_alpha.fit_transform(X_raw)
# X_cr_alpha = pd.DataFrame(
#     X_cr_alpha, columns=["time_in_hospital", "had_inpatient_days_True", "medicare_True"]
# )
# X_cr_alpha["race_AfricanAmerican"] = X_raw["race_AfricanAmerican"]

In [ ]:
#Mitigated dataset
mit_fl_cr

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,AgeCategory,Outcome
0,4.755846,142.388586,69.247356,35.807919,1.314627,32.874841,0.610972,1,1
1,-0.244154,79.388586,63.247356,29.807919,1.314627,25.874841,0.334972,1,0
2,6.755846,177.388586,61.247356,0.807919,1.314627,22.574841,0.655972,1,1
3,3.324882,99.485736,71.143711,21.490286,91.543431,29.455064,0.196952,0,0
4,-1.244154,131.388586,37.247356,35.807919,169.314627,42.374841,2.271972,1,1
...,...,...,...,...,...,...,...,...,...
672,8.755846,95.388586,73.247356,48.807919,181.314627,32.174841,0.154972,1,0
673,0.755846,116.388586,67.247356,27.807919,1.314627,36.074841,0.323972,1,0
674,3.755846,115.388586,69.247356,23.807919,113.314627,25.474841,0.228972,1,0
675,-0.244154,120.388586,57.247356,0.807919,1.314627,29.374841,0.332972,1,1


In [ ]:
mit_fl_cr.to_csv("{}/data/mitigated/mitigated-{}-fl-cr.csv".format(path_to_project,dataset_name), sep=',', index=False, encoding='utf-8')

In [ ]:
pred_fl_cr, scores_fl_cr = compute_predictions(mit_fl_cr)

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
save_predictions_scores(pred_fl_cr, scores_fl_cr, "fl-cr", dataset_name)

In [ ]:
#plot_heatmap(df, "Correlation values in the original dataset", y)

In [ ]:
#plot_heatmap(X_cr, "Correlation values after CorrelationRemover", y)

### Adversarial Mitigation uguale in AI FAIRNESS 360

In-processing: predictor changes, mitigated dataset is equal to the original one

TODO: DA CAPIRE COME USARE UN ALTRO MODELLO

In [ ]:
X = df.drop(target_variable, axis=1)
#X = X.drop(sensible_attribute, axis=1)
y = df[target_variable].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_seed, shuffle = True, test_size = 0.33)
S_train = X_train[sensible_attribute].values
X_train = X_train.drop(sensible_attribute, axis=1)

##Input data should be in 2-dimensions
pca = PCA(n_components=2)
X_train2D = pca.fit(X_train).transform(X_train)
X_test2D = pca.fit(X_test).transform(X_test)
X_2D = pca.fit(X).transform(X)

## Questo tecnica usa due tipologie di modelli, se non specificato la Logistic Regression, altrimenti NN (not-explainable)
#AM = AdversarialFairnessClassifier(predictor_model =, adversary_model = )
AM = AdversarialFairnessClassifier(random_state=random_seed)

In [ ]:
predictor = AM.fit(X_train2D, y_train, sensitive_features=S_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
pred_fl_am = predictor.predict(X_test2D)
#predictor.predict(X_2D)

In [ ]:
X_train

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction
263,8,112,72,0,0,23.6,0.840
282,4,129,60,12,231,27.5,0.527
277,2,112,68,22,94,34.1,0.315
330,0,98,82,15,84,25.2,0.299
329,12,140,82,43,325,39.2,0.528
...,...,...,...,...,...,...,...
664,1,128,88,39,110,36.5,1.057
372,2,94,68,18,76,26.0,0.561
204,1,79,80,25,37,25.4,0.583
53,2,141,58,34,128,25.4,0.699


In [ ]:
#predictor.score(X_2D, y)
predictor.score(X_test2D, y_test)

0.5

In [ ]:
X_test[target_variable] = y_test
X_test

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,AgeCategory,Outcome
308,5,0,80,32,0,41.0,0.346,1,1
70,7,83,78,26,71,29.3,0.767,1,0
502,0,121,66,30,165,34.3,0.203,1,1
470,1,77,56,30,56,33.3,1.251,0,0
285,13,152,90,33,29,26.8,0.731,1,1
...,...,...,...,...,...,...,...,...,...
627,10,94,72,18,0,23.1,0.595,1,0
364,1,143,84,23,310,42.4,1.076,0,0
382,2,139,75,0,0,25.6,0.167,1,0
24,5,109,75,26,0,36.0,0.546,1,0


In [ ]:
save_predictions_scores(pred_fl_am, None, "fl-am", dataset_name)
X_test.to_csv("{}/data/mitigated/mitigated-{}-fl-am.csv".format(path_to_project,dataset_name), sep=',', index=False, encoding='utf-8')

### Reductions Exponential Gradient (anche in AI FAIRNESS 360 [link qui](https://aif360.readthedocs.io/en/stable/modules/generated/aif360.sklearn.inprocessing.ExponentiatedGradientReduction.html))
TODO ERRORE!

In [ ]:
X = df.drop(target_variable, axis=1)
#X = X.drop(sensible_attribute, axis=1)
y = df[target_variable]#.values
S = df[sensible_attribute]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_seed, shuffle = True, test_size = 0.33)
S_train = X_train[sensible_attribute].values
#X_train = X_train.drop(sensible_attribute, axis=1)

In [ ]:
##Input data should be in 2-dimensions
pca = PCA(n_components=2)
X_train2D = pca.fit(X_train).transform(X_train)
X_test2D = pca.fit(X_test).transform(X_test)
X_2D = pca.fit(X).transform(X)

In [ ]:
mom = Moment()
#mom.load_data(X=X_train, y=y_train, sensitive_features=S)

In [ ]:
lmod = DecisionTreeClassifier(max_depth=None)
# lmod.fit(X_train2D, y_train)
# y_train_pred = lmod.predict(X_train2D)

In [ ]:
eg = ExponentiatedGradient(estimator=lmod, constraints='UtilityParity')

In [ ]:
predictor_eg = eg.fit(X_train, y_train)
#print(eg.predict(X_test))

AttributeError: ignored

### Threshold Optimizer
Post-processing: the modello chanegs (predictions), mitigated dataset is equal to the original one

Train-test split

In [ ]:
n_estimators = 30
random_seed = 1234

models = {'Logistic Regression':LogisticRegression(max_iter=500),
          'Decision Tree':DecisionTreeClassifier(max_depth=None),
          'Bagging':BaggingClassifier(DecisionTreeClassifier(max_depth=3),n_estimators=n_estimators),
          'Random Forest':RandomForestClassifier(n_estimators=n_estimators),
          'Extremely Randomized Trees':ExtraTreesClassifier(n_estimators=n_estimators),
          'Ada Boost':AdaBoostClassifier(DecisionTreeClassifier(max_depth=3),n_estimators=n_estimators)}
pred_fl_to = {}

for model_name in tqdm(models):
  X = df.drop(target_variable, axis=1)
  y = df[target_variable].values
  S = df[sensible_attribute]
  X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_seed, shuffle = True, test_size = 0.33)
  S_train = X_train[sensible_attribute]
  S_test = X_test[sensible_attribute]

  clf = models[model_name]
  cross_validate(clf,X,y,cv=StratifiedKFold(n_splits=10,shuffle=True,random_state=random_seed))

  TO = ThresholdOptimizer(estimator=clf)
  predictor_TO = TO.fit(X=X_train,y=y_train, sensitive_features=S_train)
  pred_fl_to[model_name] = predictor_TO.predict(X=X_test, sensitive_features=S_test)

  ## NO SCORE FUNCTION!

  0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/fairlearn/postprocessing/_threshold_optimizer.py:285: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  warn(
/usr/local/lib/python3.10/dist-packages/fairlearn/postprocessing/_threshold_optimizer.py:285: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  warn(
/usr/local/lib/python3.10/dist-packages/fairlearn/postprocessing/_threshold_optimizer.py:285: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.


In [ ]:
print(pred_fl_to)

{
    'Logistic Regression': array([0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0]),
    'Decision Tree': array([0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0]),
    'Bagging': array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0]),
    'Random Forest': array([0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0]),
    'Extremely Randomized Trees': array([0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, [1

In [ ]:
X_test[target_variable] = y_test
X_test.to_csv("{}/data/mitigated/mitigated-{}-fl-to.csv".format(path_to_project,dataset_name), sep=',', index=False, encoding='utf-8')

In [ ]:
save_predictions_scores(pred_fl_to, None, "fl-to", dataset_name)

In [ ]:
## OLD VERSION
#lmod = DecisionTreeClassifier(max_depth=None)
#lmod.fit(X_train, y_train)
#y_train_pred = lmod.predict(X_train)

# AIF360
[AIF360](https://aif360.mybluemix.net) is a system used in order to detect and mitigate a possible bias inside a dataset.
It requires firstly the identification of the corresponding protected attribute (sex, race, age etc.)

### Configuration

In order to use [AIF360](https://aif360.mybluemix.net) we need to define a Binary Dataset and set the privileged/unprivileged groups

In [ ]:
data_orig_aif = BinaryLabelDataset(
    favorable_label = 1,
    unfavorable_label = 0,
    df = df.copy(),
    label_names = [target_variable],
    protected_attribute_names = [sensible_attribute])

In [ ]:
privileged_groups = [{'AgeCategory': 1}]  #todo mettere sensible_attribute
unprivileged_groups = [{'AgeCategory': 0}]

## Pre-processing

### Reweighing

Pre-processing: dataset is equal to the original one, the predictor is the same, only use different weights

In [ ]:
RW = Reweighing(unprivileged_groups = unprivileged_groups,
                privileged_groups = privileged_groups)
rw_dataset = RW.fit_transform(data_orig_aif)

rw_df = rw_dataset.convert_to_dataframe()
rw_df[1]["instance_weights"][:10]

array([0.77614004, 1.16634653, 0.77614004, 0.78957164, 0.77614004,
       1.16634653, 0.77614004, 1.16634653, 0.77614004, 1.16634653])

In [ ]:
scores = {}
predicted_values = {}
for model_name in models:
    X = df.drop(target_variable, axis=1)
    y = df[target_variable].values
    clf = models[model_name]
    cross_val_results = cross_val_predict(clf, X, y, cv=StratifiedKFold(n_splits=10,shuffle=True,random_state=random_seed), fit_params={'sample_weight': rw_df[1]["instance_weights"]})
    predicted_values[model_name] = cross_val_results

    score = cross_val_score(clf,X,y,cv=StratifiedKFold(n_splits=10,shuffle=True,random_state=random_seed))
    scores[model_name] = [round(np.average(score), 5), round(np.std(score), 5)]

In [ ]:
save_predictions_scores(predicted_values, scores, "aif360-rw", dataset_name)

### Disparate Impact Remover

Pre-processing: It transforms the dataset in a new mitigated one

In [ ]:
!pip install BlackBoxAuditing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for BlackBoxAuditing: filename=BlackBoxAuditing-0.1.54-py2.py3-none-any.whl size=1394755 sha256=67951dd8c849dbccbdadeab3ad8df6a3ea1f7b6a7c66ffcb3bed2b1db9ff90dd
  Stored in directory: /root/.cache/pip/wheels/c0/4f/b1/80e1b0790df07536470758fe0a4f9ff8fa942fd9fe30bbb192
Successfully built BlackBoxAuditing


In [ ]:
#levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
DIR = DisparateImpactRemover(repair_level=0.5)
train_repd = DIR.fit_transform(data_orig_aif)
mit_aif360_di = train_repd.convert_to_dataframe()[0]

X = mit_aif360_di.drop(target_variable, axis=1)
y = mit_aif360_di[target_variable].values
pred_aif360_di = {}
scores_aif360_di= {}

for model_name in tqdm(models):
  clf = models[model_name]
  cross_val_results = cross_val_predict(clf, X, y, cv=StratifiedKFold(n_splits=10,shuffle=True,random_state=random_seed))
  pred_aif360_di[model_name] = cross_val_results

  score = cross_val_score(clf, X,y,cv=StratifiedKFold(n_splits=10,shuffle=True,random_state=random_seed))
  scores_aif360_di[model_name] = [round(np.average(score), 5), round(np.std(score), 5)]

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
mit_aif360_di.to_csv("{}/data/mitigated/mitigated-{}-aif360-di.csv".format(path_to_project,dataset_name), sep=',', index=False, encoding='utf-8')
save_predictions_scores(pred_aif360_di, scores_aif360_di, "aif360-di", dataset_name)

### Learning Fair Representation

Pre-processing: mitigated dataset
WARN: PREDIZIONI QUASI TUTTE UGUALI!!!

In [ ]:
TR = LFR(unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    seed= random_seed, k=10, Ax=0.1, Ay=1.0, Az=50.0,
    verbose=1)

dataset_orig_train, dataset_orig_test = data_orig_aif.split([0.7], shuffle=True)
TR = TR.fit(dataset_orig_train, maxiter=5000, maxfun=5000)
transf_dataset = TR.transform(data_orig_aif)
mit_aif360_lfr = transf_dataset.convert_to_dataframe()[0]

step: 0, loss: 824.9285750274747, L_x: 8240.480048360514,  L_y: 0.7896113500134818,  L_z: 0.0018191768281961285
step: 250, loss: 818.1052393552042, L_x: 8173.329026782085,  L_y: 0.6822309251483929,  L_z: 0.0018021150369468704
step: 500, loss: 778.3381924748272, L_x: 7775.038326648103,  L_y: 0.6370734727996723,  L_z: 0.003945726744344051
step: 750, loss: 646.4552637667675, L_x: 6417.462074783967,  L_y: 3.799338017858711,  L_z: 0.018194365410240894
step: 1000, loss: 489.8594434978338, L_x: 4862.610739388296,  L_y: 2.304022437206225,  L_z: 0.02588694243595894
step: 1250, loss: 881.3134976366682, L_x: 8804.295279289721,  L_y: 0.8539064033071836,  L_z: 0.0006012660877768113
step: 1500, loss: 142.54720128437035, L_x: 1403.864087509763,  L_y: 1.6957063750357926,  L_z: 0.009301723167165033
step: 1750, loss: 142.5347664678018, L_x: 1407.0333531901615,  L_y: 1.3735991026377379,  L_z: 0.00915664092295826
step: 2000, loss: 149.30340692860042, L_x: 1418.706756628542,  L_y: 6.880178266010017,  L_z: 

In [ ]:
mit_aif360_lfr

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,AgeCategory,Outcome
0,3.516225,114.419342,71.777841,14.556200,8.450839,31.378284,0.416086,1.0,0.0
1,3.516225,114.419342,71.777841,14.556200,8.450839,31.378284,0.416086,1.0,0.0
2,3.516225,114.419342,71.777841,14.556200,8.450839,31.378284,0.416086,1.0,0.0
3,2.773101,124.990581,69.886068,31.363408,150.770936,32.700192,0.472150,0.0,1.0
4,2.773101,124.990581,69.886068,31.363408,150.770936,32.700192,0.472150,1.0,1.0
...,...,...,...,...,...,...,...,...,...
672,2.773101,124.990581,69.886068,31.363408,150.770936,32.700192,0.472150,1.0,1.0
673,3.516225,114.419342,71.777841,14.556200,8.450839,31.378284,0.416086,1.0,0.0
674,2.773101,124.990581,69.886068,31.363408,150.770936,32.700192,0.472150,1.0,1.0
675,3.516225,114.419342,71.777841,14.556200,8.450839,31.378284,0.416086,1.0,0.0


In [ ]:
X = mit_aif360_lfr.drop(target_variable, axis=1)
y = mit_aif360_lfr[target_variable].values
pred_aif360_lfr = {}
scores_aif360_lfr= {}

for model_name in tqdm(models):
  clf = models[model_name]
  cross_val_results = cross_val_predict(clf, X, y, cv=StratifiedKFold(n_splits=10,shuffle=True,random_state=random_seed))
  pred_aif360_lfr[model_name] = cross_val_results

  score = cross_val_score(clf, X,y,cv=StratifiedKFold(n_splits=10,shuffle=True,random_state=random_seed))
  scores_aif360_lfr[model_name] = [round(np.average(score), 5), round(np.std(score), 5)]

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
#predizioni e scores tutti uguali!!
print(pred_aif360_lfr)
print(scores_aif360_lfr)

{
    'Logistic Regression': array([0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1.,
       0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0.,
       1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0.,
       1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0.,
       0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0.,
       0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0.,
       0., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0.,
       0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0.,
       0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0.,
       1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1.,
       0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0.,
       1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1.,
       1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0.,
       1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
       1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.]),
    'Decision Tree': array([0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1.,
       0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0.,
       1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0.,
       1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0.,
       0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0.,
       0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0

{
    'Logistic Regression': [0.99853, 0.00441],
    'Decision Tree': [0.99853, 0.00441],
    'Bagging': [0.99853, 0.00441],
    'Random Forest': [0.99853, 0.00441],
    'Extremely Randomized Trees': [0.99853, 0.00441],
    'Ada Boost': [0.99853, 0.00441]
}

In [ ]:
mit_aif360_lfr.to_csv("{}/data/mitigated/mitigated-{}-aif360-lfr.csv".format(path_to_project,dataset_name), sep=',', index=False, encoding='utf-8')
save_predictions_scores(pred_aif360_lfr, scores_aif360_lfr, "aif360-lfr", dataset_name)

### Optimized Preprocessing

Pre-processing: mitigated dataset
Richiede valori binari come input quindi è stato fatto un pre-processing sui valori per essere 0,1

TODO:Errore nel transform
Vecchio: OptTool difficile da creare, chiesto a Daniel delucidazioni

In [ ]:
df2=df.drop(columns=['Pregnancies','DiabetesPedigreeFunction'])
#df2 = df.copy()
df2['BMI'] = np.where(df2['BMI'].between(18,25), 1, 0) # 1 valore giusto, 0 anomalo
df2['Glucose'] = np.where(df2['Glucose'] < 120, 1, 0)
df2["BloodPressure"]=np.where(df2["BloodPressure"].between(60,80),1,0)
df2["SkinThickness"]=np.where(df2["SkinThickness"].between(22,24),1,0)
df2["Insulin"]=np.where(df2["Insulin"].between(5,25),1,0)

data_orig_aif_op = BinaryLabelDataset(
    favorable_label = 1,
    unfavorable_label = 0,
    df = df2.copy(),
    label_names = [target_variable],
    protected_attribute_names = [sensible_attribute],
    metadata = default_mappings)

In [ ]:
df2.head(5)

,Glucose,BloodPressure,SkinThickness,Insulin,BMI,Outcome,AgeCategory
0,0,1,0,0,0,1,1
1,1,1,0,0,0,0,1
2,0,1,0,0,1,1,1
3,1,1,1,0,0,0,0
4,0,0,0,0,0,1,1


In [ ]:
def get_distortion_diabetes(vold,vnew):
        bad_val=3
        OutNew=vnew["Outcome"]
        OutOld=vold["Outcome"]
        GluNew=vnew["Glucose"]
        GluOld=vold["Glucose"]
        InsOld=vold["Insulin"]
        InsNew=vnew["Insulin"]

        if ((OutNew>OutOld)& (InsNew<InsOld)): #| ((OutNew>OutOld) & (GluNew<GluOld)):
            return bad_val
        else:
            return 0

In [ ]:
optim_options = {
    "distortion_fun": get_distortion_diabetes,
    "epsilon": 0.05,
    "clist": [0.99, 1.99, 2.99],
    "dlist": [.1, 0.05, 0]
}

OP = OptimPreproc(OptTools, optim_options)
dataset_orig_train, dataset_orig_test = data_orig_aif_op.split([0.7], shuffle=True)
dataset_orig_train

               instance weights features                                      \
                                                                               
                                 Glucose BloodPressure SkinThickness Insulin   
instance names                                                                 
308                         1.0      1.0           1.0           0.0     0.0   
70                          1.0      1.0           1.0           0.0     0.0   
502                         1.0      0.0           1.0           0.0     0.0   
470                         1.0      1.0           0.0           0.0     0.0   
285                         1.0      0.0           0.0           0.0     0.0   
...                         ...      ...           ...           ...     ...   
565                         1.0      1.0           0.0           0.0     0.0   
488                         1.0      0.0           1.0           0.0     0.0   
216                         1.0      0.0

In [ ]:
OP= OP.fit(dataset_orig_train)

/usr/local/lib/python3.10/dist-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 2 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/

Optimized Preprocessing: Objective converged to 0.000000


In [ ]:
transf_dataset = OP.transform(data_orig_aif_op, transform_Y=True)
mit_aif360_op = transf_dataset.convert_to_dataframe()[0]
X = mit_aif360_op.drop(target_variable, axis=1)
y = mit_aif360_op[target_variable].values
pred_aif360_op = {}
scores_aif360_op= {}

for model_name in tqdm(models):
  clf = models[model_name]
  cross_val_results = cross_val_predict(clf, X, y, cv=StratifiedKFold(n_splits=10,shuffle=True,random_state=random_seed))
  pred_aif360_op[model_name] = cross_val_results

  score = cross_val_score(clf, X,y,cv=StratifiedKFold(n_splits=10,shuffle=True,random_state=random_seed))
  scores_aif360_op[model_name] = [round(np.average(score), 5), round(np.std(score), 5)]

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
## extra code
# dataset_transf_train = OP.transform(dataset_orig_train, transform_Y=True)
# OP = OP.fit(dataset_orig_train)
# dataset_transf_train = OP.transform(dataset_orig_train, transform_Y=False)
# dataset_transf_train = dataset_orig_train.align_datasets(dataset_transf_train)

In [ ]:
mit_aif360_op.to_csv("{}/data/mitigated/mitigated-{}-aif360-op.csv".format(path_to_project,dataset_name), sep=',', index=False, encoding='utf-8')
save_predictions_scores(pred_aif360_op, scores_aif360_op, "aif360-op", dataset_name)

##In-processing

Ogni tecnica genera un nuovo classificatore: no score and no mitigated dataset

### Adversarial Debiasing UGUALE A FAIRLEARN (ORIGINALE)
In-processing: dataset is equal to the original one, the predictor is fairer by construction and given

NO SCORE FUNCTION

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
sess = tf.Session()
AD = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess)
dataset_orig_train, dataset_orig_test = data_orig_aif.split([0.7], shuffle=True)

# This technique uses two types of ML models: Neural Networks (NNs) and Logistic Regression (LR) (default), we chose LR because NN are not-explainable
#AM = AdversarialFairnessClassifier(predictor_model =, adversary_model = )
AM = AdversarialFairnessClassifier(random_state=random_seed)
AD.fit(dataset_orig_train)
predictions = AD.predict(data_orig_aif)

In [ ]:
mit_aif360_ad = predictions.convert_to_dataframe()[0]
#mit_aif360_is the same dataset with predictions in target_variable column
pred_aif360_ad = mit_aif360_ad[target_variable]
print(pred_aif360_ad)

0      1.0
1      0.0
2      1.0
3      0.0
4      1.0
      ... 
672    0.0
673    0.0
674    0.0
675    0.0
676    0.0
Name: Outcome, Length: 677, dtype: float64

In [ ]:
save_predictions_scores(pred_aif360_ad.to_numpy(), None, "aif360-ad", dataset_name)

### Prejudice remover
In-processing: dataset is equal to the original one, the predictor is fairer by construction and given

NO SCORE FUNCTION


In [ ]:
PR = PrejudiceRemover(eta=1.0, sensitive_attr="AgeCategory", class_attr="Outcome")
dataset_orig_train, dataset_orig_test = data_orig_aif.split([0.7], shuffle=True)
PR.fit(dataset_orig_train)
#mit_aif360_pr the same dataset with predictions in target_variable column
predictions = PR.predict(data_orig_aif)
mit_aif360_pr = predictions.convert_to_dataframe()[0]
print(mit_aif360_pr)
pred_aif360_pr = mit_aif360_pr[target_variable]
print(pred_aif360_pr)

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6.0    148.0           72.0           35.0      0.0  33.6   
1            1.0     85.0           66.0           29.0      0.0  26.6   
2            8.0    183.0           64.0            0.0      0.0  23.3   
3            1.0     89.0           66.0           23.0     94.0  28.1   
4            0.0    137.0           40.0           35.0    168.0  43.1   
..           ...      ...            ...            ...      ...   ...   
672         10.0    101.0           76.0           48.0    180.0  32.9   
673          2.0    122.0           70.0           27.0      0.0  36.8   
674          5.0    121.0           72.0           23.0    112.0  26.2   
675          1.0    126.0           60.0            0.0      0.0  30.1   
676          1.0     93.0           70.0           31.0      0.0  30.4   

     DiabetesPedigreeFunction  AgeCategory  Outcome  
0                       0.627          1.0      1.0  
1                       0.351          1.0      0.0  
2                       0.672          1.0      1.0  
3                       0.167          0.0      0.0  
4                       2.288          1.0      1.0  
..                        ...          ...      ...  
672                     0.171          1.0      0.0  
673                     0.340          1.0      0.0  
674                     0.245          1.0      0.0  
675                     0.349          1.0      0.0  
676                     0.315          0.0      0.0  

[677 rows x 9 columns]

0      1.0
1      0.0
2      1.0
3      0.0
4      1.0
      ... 
672    0.0
673    0.0
674    0.0
675    0.0
676    0.0
Name: Outcome, Length: 677, dtype: float64

In [ ]:
save_predictions_scores(pred_aif360_pr.to_numpy(), None, "aif360-pr", dataset_name)

### Exponentiated Gradient Reduction COSA NON VA?
In-processing: dataset is equal to the original one, the predictor is fairer by construction and given

NO SCORE FUNCTION

In [ ]:
X = df.drop(target_variable, axis=1)
# X = X.drop(sensible_attribute, axis=1)
y = df[target_variable].values
S = df[sensible_attribute]
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_seed, shuffle = True, test_size = 0.33)
# S_train = X_train[sensible_attribute].values
# X_train = X_train.drop(sensible_attribute, axis=1)

# Input data should be in 2-dimensions
# pca = PCA(n_components=2)
# X_train2D = pca.fit(X_train).transform(X_train)
# X_test2D = pca.fit(X_test).transform(X_test)
# X_2D = pca.fit(X).transform(X)
dataset_orig_train, dataset_orig_test = data_orig_aif.split([0.7], shuffle=True)
lmod = DecisionTreeClassifier(max_depth=None)

X_test = dataset_orig_test.features
y_test = dataset_orig_test.labels.ravel()
X_train = dataset_orig_train.features
y_train = dataset_orig_train.labels.ravel()

#lmod.fit(X_train, y_train)
#y_pred = lmod.predict(X_test)

mom = Moment()
#mom.load_data(X=X_train, y=y_train, sensitive_features=S)

In [ ]:
dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
dataset_orig_test_pred.labels = y_pred

In [ ]:
dataset_orig_test

               instance weights    features                        \
                                                                    
                                Pregnancies Glucose BloodPressure   
instance names                                                      
532                         1.0         1.0   181.0          78.0   
264                         1.0         2.0   144.0          58.0   
358                         1.0         2.0   123.0          48.0   
508                         1.0         6.0   108.0          44.0   
227                         1.0         2.0   114.0          68.0   
...                         ...         ...     ...           ...   
664                         1.0         1.0   128.0          88.0   
372                         1.0         2.0    94.0          68.0   
204                         1.0         1.0    79.0          80.0   
53                          1.0         2.0   141.0          58.0   
294                         1.0   

In [ ]:
exp_grad_red = ExponentiatedGradientReduction(estimator=lmod, constraints=mom, drop_prot_attr=False)

In [ ]:
exp_grad_red = ExponentiatedGradient(estimator=lmod, constraints=mom)

In [ ]:
exp_grad_red.fit(dataset_orig_train, y_train)
#exp_grad_red_pred = exp_grad_red.predict(dataset_orig_test, random_state=random_seed)

AttributeError: 'Moment' object has no attribute 'default_objective'

In [ ]:
#exp_grad_red.fit(dataset_orig_train)
#exp_grad_red_pred = exp_grad_red.predict(dataset_orig_test)
#exp_grad_red_pred = exp_grad_red.predict(dataset_orig_test_pred)

TypeError: ExponentiatedGradientReduction.fit() takes 2 positional arguments but 3 were given

In [ ]:
lmod = DecisionTreeClassifier(max_depth=None)

exp_grad_red = ExponentiatedGradientReduction(estimator=lmod, constraints=mom, drop_prot_attr=False)

# exp_grad_red.fit(dataset_orig_train)
lmod.fit(X_train, y_train)

X_test = dataset_orig_test.features
y_test = dataset_orig_test.labels.ravel()

y_pred = lmod.predict(X_test)
dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
dataset_orig_test_pred.labels = y_pred

In [ ]:
dataset_orig_test_pred

ValueError: ignored

In [ ]:
exp_grad_red_pred = exp_grad_red.predict(dataset_orig_test_pred)

## Post-processing

In the first phase there is the model building: the same for Calibrated Equalized Post-processing, Equalize Odds Post_processing and Reject Option Classification

####EOPP non compila!


In [ ]:
def build_model_and_predictionsOLD(model, data_orig_aif):
  # Get the dataset and split into train and test
  dataset_orig_train, dataset_orig_vt = data_orig_aif.split([0.5], shuffle=True)
  dataset_orig_valid, dataset_orig_test = data_orig_aif.split([0.5], shuffle=True)

  # Logistic regression classifier and predictions
  scale_orig = StandardScaler()
  X_train = scale_orig.fit_transform(dataset_orig_train.features)
  y_train = dataset_orig_train.labels.ravel()

  model.fit(X_train, y_train)
  y_train_pred = model.predict(X_train)

  # positive class index
  pos_ind = np.where(model.classes_ == dataset_orig_train.favorable_label)[0][0]

  dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
  dataset_orig_train_pred.labels = y_train_pred

  dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
  X_valid = scale_orig.transform(dataset_orig_valid_pred.features)
  y_valid = dataset_orig_valid_pred.labels
  dataset_orig_valid_pred.scores = model.predict_proba(X_valid)[:,pos_ind].reshape(-1,1)

  #dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
  #X_test = scale_orig.transform(dataset_orig_test_pred.features)
  #y_test = dataset_orig_test_pred.labels
  #dataset_orig_test_pred.scores = model.predict_proba(X_test)[:,pos_ind].reshape(-1,1)
  print("VALID")
  print(dataset_orig_valid)
  print("VALID_pred")
  print(dataset_orig_valid_pred)
  print("TEST")
  print(dataset_orig_test)

  return dataset_orig_valid, dataset_orig_valid_pred

In [ ]:
def build_model_and_predictions(model, data_orig_aif):
  # Get the dataset and split into train and test
  dataset_orig_train, dataset_orig_vt = data_orig_aif.split([0.5], shuffle=True, seed=random_seed)
  dataset_orig_valid, dataset_orig_test = data_orig_aif.split([0.5], shuffle=True, seed=random_seed)

  dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
  dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
  dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)

  # Logistic regression classifier and predictions

  X_train = dataset_orig_train.features
  y_train = dataset_orig_train.labels.ravel()

  model.fit(X_train, y_train)
  y_train_pred = model.predict(X_train)

  # positive class index
  pos_ind = np.where(model.classes_ == dataset_orig_train.favorable_label)[0][0]

  dataset_orig_train_pred.labels = y_train_pred

  X_valid = dataset_orig_valid_pred.features
  y_valid = dataset_orig_valid_pred.labels
  dataset_orig_valid_pred.scores = model.predict_proba(X_valid)[:,pos_ind].reshape(-1,1)

  #dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
  #X_test = scale_orig.transform(dataset_orig_test_pred.features)
  #y_test = dataset_orig_test_pred.labels
  #dataset_orig_test_pred.scores = model.predict_proba(X_test)[:,pos_ind].reshape(-1,1)

  return dataset_orig_valid, dataset_orig_valid_pred, dataset_orig_test

### Calibrated Equalized Post-Processing

Post-processing: model's labels are changes, mitigated dataset is equal to the original one

NO EMBEDDED SCORE FUNCTION

In [ ]:
CEPP = CalibratedEqOddsPostprocessing(unprivileged_groups= unprivileged_groups, privileged_groups = privileged_groups, cost_constraint='weighted', seed=random_seed)
predicted_values = {}

# we adopt different models but they retrieve the same predictions applyting CEPP technique
for model_name in tqdm(models):
  dataset_orig_valid, dataset_orig_valid_pred, dataset_orig_test = build_model_and_predictions(models[model_name], data_orig_aif)
  cpp = CEPP.fit(dataset_orig_valid, dataset_orig_valid_pred)
  #predictions = cpp.predict(data_orig_aif)
  predictions = cpp.predict(dataset_orig_test)
  mit_aif360_cpp = predictions.convert_to_dataframe()[0]
  pred_aif360_cpp = mit_aif360_cpp[target_variable]
  predicted_values[model_name] = pred_aif360_cpp.to_numpy()
  print(model_name, pred_aif360_cpp[:10])

  ##EXTRA
  #dataset_transf_valid_pred = cpp.predict(dataset_orig_valid_pred)
  #dataset_transf_test_pred = cpp.predict(dataset_orig_test_pred)
  #print(cpp.predict(data_orig_aif))

  0%|          | 0/6 [00:00<?, ?it/s]

Logistic Regression 621    0.0
134    1.0
432    0.0
462    0.0
545    0.0
672    0.0
539    0.0
280    0.0
550    0.0
612    1.0
Name: Outcome, dtype: float64

Decision Tree 621    1.0
134    1.0
432    0.0
462    0.0
545    0.0
672    0.0
539    0.0
280    0.0
550    0.0
612    1.0
Name: Outcome, dtype: float64

Bagging 621    0.0
134    1.0
432    0.0
462    0.0
545    0.0
672    0.0
539    0.0
280    0.0
550    0.0
612    1.0
Name: Outcome, dtype: float64

Random Forest 621    0.0
134    1.0
432    0.0
462    0.0
545    0.0
672    0.0
539    0.0
280    0.0
550    0.0
612    1.0
Name: Outcome, dtype: float64

Extremely Randomized Trees 621    1.0
134    1.0
432    0.0
462    0.0
545    0.0
672    0.0
539    0.0
280    0.0
550    0.0
612    1.0
Name: Outcome, dtype: float64

Ada Boost 621    0.0
134    1.0
432    0.0
462    0.0
545    0.0
672    0.0
539    0.0
280    0.0
550    0.0
612    1.0
Name: Outcome, dtype: float64

In [ ]:
save_predictions_scores(predicted_values, None, "aif360-cepp", dataset_name)

### Equalize Odds Post-Processing
Post-processing: model's labels are changes, mitigated dataset is equal to the original one

NO EMBEDDED SCORE FUNCTION

In [ ]:
def build_model_and_predictionsEOPP(model, data_orig_aif):
  # Get the dataset and split into train and test
  dataset_orig_train, dataset_orig_vt = data_orig_aif.split([0.6], shuffle=True)
  dataset_orig_valid, dataset_orig_test = data_orig_aif.split([0.5], shuffle=True)

  # Placeholder for predicted and transformed datasets
  dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
  dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
  dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)

  dataset_new_valid_pred = dataset_orig_valid.copy(deepcopy=True)
  dataset_new_test_pred = dataset_orig_test.copy(deepcopy=True)

  # Logistic regression classifier and predictions for training data
  scale_orig = StandardScaler()
  X_train = scale_orig.fit_transform(dataset_orig_train.features)
  y_train = dataset_orig_train.labels.ravel()
  lmod = model
  lmod.fit(X_train, y_train)

  fav_idx = np.where(lmod.classes_ == dataset_orig_train.favorable_label)[0][0]
  y_train_pred_prob = lmod.predict_proba(X_train)[:,fav_idx]

  # Prediction probs for validation and testing data
  X_valid = scale_orig.transform(dataset_orig_valid.features)
  y_valid_pred_prob = lmod.predict_proba(X_valid)[:,fav_idx]

  X_test = scale_orig.transform(dataset_orig_test.features)
  y_test_pred_prob = lmod.predict_proba(X_test)[:,fav_idx]

  class_thresh = 0.5
  dataset_orig_train_pred.scores = y_train_pred_prob.reshape(-1,1)
  dataset_orig_valid_pred.scores = y_valid_pred_prob.reshape(-1,1)
  dataset_orig_test_pred.scores = y_test_pred_prob.reshape(-1,1)

  y_train_pred = np.zeros_like(dataset_orig_train_pred.labels)
  y_train_pred[y_train_pred_prob >= class_thresh] = dataset_orig_train_pred.favorable_label
  y_train_pred[~(y_train_pred_prob >= class_thresh)] = dataset_orig_train_pred.unfavorable_label
  dataset_orig_train_pred.labels = y_train_pred

  y_valid_pred = np.zeros_like(dataset_orig_valid_pred.labels)
  y_valid_pred[y_valid_pred_prob >= class_thresh] = dataset_orig_valid_pred.favorable_label
  y_valid_pred[~(y_valid_pred_prob >= class_thresh)] = dataset_orig_valid_pred.unfavorable_label
  dataset_orig_valid_pred.labels = y_valid_pred

  y_test_pred = np.zeros_like(dataset_orig_test_pred.labels)
  y_test_pred[y_test_pred_prob >= class_thresh] = dataset_orig_test_pred.favorable_label
  y_test_pred[~(y_test_pred_prob >= class_thresh)] = dataset_orig_test_pred.unfavorable_label
  dataset_orig_test_pred.labels = y_test_pred
  return dataset_orig_valid, dataset_orig_valid_pred, dataset_orig_test

In [ ]:
EOPP = EqOddsPostprocessing(unprivileged_groups= unprivileged_groups, privileged_groups = privileged_groups, seed=random_seed)

predicted_values = {}
for model_name in tqdm(models):
  dataset_orig_valid, dataset_orig_valid_pred, dataset_orig_test = build_model_and_predictionsEOPP(models[model_name], data_orig_aif)
  eopp = EOPP.fit(dataset_orig_valid, dataset_orig_valid_pred)
  predictions = eopp.predict(dataset_orig_test)
  mit_aif360_eopp = predictions.convert_to_dataframe()[0]
  pred_aif360_eopp = mit_aif360_eopp[target_variable]
  predicted_values[model_name] = pred_aif360_eopp.to_numpy()
  print(model_name, pred_aif360_eopp[10])

  0%|          | 0/6 [00:00<?, ?it/s]

TypeError: No loop matching the specified signature and casting was found for ufunc logical_and

In [ ]:
EOPP = EqOddsPostprocessing(unprivileged_groups= unprivileged_groups, privileged_groups = privileged_groups, seed=random_seed)

predicted_values = {}
for model_name in tqdm(models):
  dataset_orig_valid, dataset_orig_valid_pred, dataset_orig_test = build_model_and_predictions(models[model_name], data_orig_aif)
  eopp = EOPP.fit(dataset_orig_valid, dataset_orig_valid_pred)
  predictions = eopp.predict(dataset_orig_test)
  mit_aif360_eopp = predictions.convert_to_dataframe()[0]
  pred_aif360_eopp = mit_aif360_eopp[target_variable]
  predicted_values[model_name] = pred_aif360_eopp.to_numpy()
  print(model_name, pred_aif360_eopp[10])

  ##EXTRA
  #EOPP = EqOddsPostprocessing(unprivileged_groups= unprivileged_groups, privileged_groups = privileged_groups, seed=random_seed)
  #eopp = EOPP.fit(dataset_orig_valid, dataset_orig_valid_pred)
  #dataset_transf_valid_pred = eopp.predict(dataset_orig_valid_pred)
  #dataset_transf_test_pred = eopp.predict(dataset_orig_test_pred)
  #print(eopp.predict(data_orig_aif))

  0%|          | 0/6 [00:00<?, ?it/s]

VALID

instance weights    features                        \
                                                                    
                                Pregnancies Glucose BloodPressure   
instance names                                                      
90                          1.0         1.0    81.0          72.0   
668                         1.0         6.0   190.0          92.0   
246                         1.0         0.0   146.0          70.0   
9                           1.0        10.0   139.0          80.0   
213                         1.0         4.0    91.0          70.0   
...                         ...         ...     ...           ...   
594                         1.0         6.0   195.0          70.0   
513                         1.0        12.0   121.0          78.0   
210                         1.0         9.0   164.0          84.0   
380                         1.0         3.0    89.0          74.0   
94                          1.0         3.0    83.0          58.0   

                                                                     \
                                                                      
               SkinThickness Insulin   BMI DiabetesPedigreeFunction   
instance names                                                        
90                      18.0    40.0  26.6                    0.283   
668                      0.0     0.0  35.5                    0.278   
246                      0.0     0.0  37.9                    0.334   
9                        0.0     0.0  27.1                    1.441   
213                     32.0    88.0  33.1                    0.446   
...                      ...     ...   ...                      ...   
594                      0.0     0.0  30.9                    0.328   
513                     17.0     0.0  26.5                    0.259   
210                     21.0     0.0  30.8                    0.831   
380                     16.0    85.0  30.4                    0.551   
94                      31.0    18.0  34.3                    0.336   

                                   labels  
               protected attribute         
                       AgeCategory         
instance names                             
90                             0.0    0.0  
668                            1.0    1.0  
246                            1.0    1.0  
9                              1.0    0.0  
213                            0.0    0.0  
...                            ...    ...  
594                            1.0    1.0  
513                            1.0    0.0  
210                            1.0    1.0  
380                            1.0    0.0  
94                             0.0    0.0  

[338 rows x 10 columns]

VALID_pred

instance weights    features                        \
                                                                    
                                Pregnancies Glucose BloodPressure   
instance names                                                      
90                          1.0         1.0    81.0          72.0   
668                         1.0         6.0   190.0          92.0   
246                         1.0         0.0   146.0          70.0   
9                           1.0        10.0   139.0          80.0   
213                         1.0         4.0    91.0          70.0   
...                         ...         ...     ...           ...   
594                         1.0         6.0   195.0          70.0   
513                         1.0        12.0   121.0          78.0   
210                         1.0         9.0   164.0          84.0   
380                         1.0         3.0    89.0          74.0   
94                          1.0         3.0    83.0          58.0   

                                                                     \
                                                                      
               SkinThickness Insulin   BMI DiabetesPedigreeFunction   
instance names                                                        
90                      18.0    40.0  26.6                    0.283   
668                      0.0     0.0  35.5                    0.278   
246                      0.0     0.0  37.9                    0.334   
9                        0.0     0.0  27.1                    1.441   
213                     32.0    88.0  33.1                    0.446   
...                      ...     ...   ...                      ...   
594                      0.0     0.0  30.9                    0.328   
513                     17.0     0.0  26.5                    0.259   
210                     21.0     0.0  30.8                    0.831   
380                     16.0    85.0  30.4                    0.551   
94                      31.0    18.0  34.3                    0.336   

                                   labels  
               protected attribute         
                       AgeCategory         
instance names                             
90                             0.0    0.0  
668                            1.0    1.0  
246                            1.0    1.0  
9                              1.0    0.0  
213                            0.0    0.0  
...                            ...    ...  
594                            1.0    1.0  
513                            1.0    0.0  
210                            1.0    1.0  
380                            1.0    0.0  
94                             0.0    0.0  

[338 rows x 10 columns]

TEST

instance weights    features                        \
                                                                    
                                Pregnancies Glucose BloodPressure   
instance names                                                      
384                         1.0        12.0   140.0          85.0   
532                         1.0         1.0   181.0          78.0   
188                         1.0         7.0   179.0          95.0   
144                         1.0         0.0   131.0          88.0   
113                         1.0         9.0   122.0          56.0   
...                         ...         ...     ...           ...   
483                         1.0         1.0   116.0          70.0   
404                         1.0        10.0   148.0          84.0   
452                         1.0         2.0    91.0          62.0   
171                         1.0         5.0   158.0          84.0   
38                          1.0         0.0   180.0          66.0   

                                                                     \
                                                                      
               SkinThickness Insulin   BMI DiabetesPedigreeFunction   
instance names                                                        
384                     33.0     0.0  37.4                    0.244   
532                     42.0   293.0  40.0                    1.258   
188                     31.0     0.0  34.2                    0.164   
144                      0.0     0.0  31.6                    0.743   
113                      0.0     0.0  33.3                    1.114   
...                      ...     ...   ...                      ...   
483                     28.0     0.0  27.4                    0.204   
404                     48.0   237.0  37.6                    1.001   
452                      0.0     0.0  27.3                    0.525   
171                     41.0   210.0  39.4                    0.395   
38                      39.0     0.0  42.0                    1.893   

                                   labels  
               protected attribute         
                       AgeCategory         
instance names                             
384                            1.0    0.0  
532                            0.0    1.0  
188                            1.0    0.0  
144                            1.0    1.0  
113                            1.0    1.0  
...                            ...    ...  
483                            0.0    0.0  
404                            1.0    1.0  
452                            0.0    0.0  
171                            1.0    1.0  
38                             0.0    1.0  

[339 rows x 10 columns]

TypeError: No loop matching the specified signature and casting was found for ufunc logical_and

In [ ]:
save_predictions_scores(predicted_values, None, "aif360-eopp", dataset_name)

### Reject Option Classification

Post-processing: model's labels are changes, mitigated dataset is equal to the original one

NO EMBEDDED SCORE FUCTION

In [ ]:
ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups,
                                 privileged_groups=privileged_groups,
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                  num_class_thresh=100, num_ROC_margin=50,
                                  metric_name="Statistical parity difference", metric_ub=0.05, metric_lb=-0.05)
predicted_values = {}
for model_name in tqdm(models):
  dataset_orig_valid, dataset_orig_valid_pred, dataset_orig_test = build_model_and_predictions(models[model_name], data_orig_aif)
  roc = ROC.fit(dataset_orig_valid, dataset_orig_valid_pred)
  predictions = roc.predict(dataset_orig_test)
  mit_aif360_roc = predictions.convert_to_dataframe()[0]
  pred_aif360_roc = mit_aif360_roc[target_variable]
  predicted_values[model_name] = pred_aif360_roc.to_numpy()
  print(model_name, '\n', predicted_values[model_name][:10])

  0%|          | 0/6 [00:00<?, ?it/s]

Logistic Regression 
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 1.]

/usr/local/lib/python3.10/dist-packages/aif360/algorithms/postprocessing/reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")


Decision Tree 
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 1.]

Bagging 
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 1.]

Random Forest 
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 1.]

/usr/local/lib/python3.10/dist-packages/aif360/algorithms/postprocessing/reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")


Extremely Randomized Trees 
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 1.]

Ada Boost 
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 1.]

In [ ]:
predicted_values

{'Logistic Regression': array([1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1.,
        0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1.,
        1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
        0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0.,
        0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0.,
        1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
        1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0.,
        1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0.,
        0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0

In [ ]:
save_predictions_scores(predicted_values, None, "aif360-roc", dataset_name)

Extra code

In [ ]:
## EXTRA CODE TO CHECK
dataset_orig_train, dataset_orig_vt = data_orig_aif.split([0.7], shuffle=True)
dataset_orig_valid, dataset_orig_test = data_orig_aif.split([0.5], shuffle=True)
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train,
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

metric_orig_valid = BinaryLabelDatasetMetric(dataset_orig_valid,
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Original validation dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_valid.mean_difference())

metric_orig_test = BinaryLabelDatasetMetric(dataset_orig_test,
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Original test dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.271225

#### Original validation dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.267612

#### Original test dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.278761

In [ ]:
# Placeholder for predicted and transformed datasets
dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)

dataset_new_valid_pred = dataset_orig_valid.copy(deepcopy=True)
dataset_new_test_pred = dataset_orig_test.copy(deepcopy=True)

# Logistic regression classifier and predictions for training data
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(dataset_orig_train.features)
y_train = dataset_orig_train.labels.ravel()

## TODO Try with different models
lmod = LogisticRegression()
lmod.fit(X_train, y_train)

fav_idx = np.where(lmod.classes_ == dataset_orig_train.favorable_label)[0][0]
y_train_pred_prob = lmod.predict_proba(X_train)[:,fav_idx]

# Prediction probs for validation and testing data
X_valid = scale_orig.transform(dataset_orig_valid.features)
y_valid_pred_prob = lmod.predict_proba(X_valid)[:,fav_idx]

X_test = scale_orig.transform(dataset_orig_test.features)
y_test_pred_prob = lmod.predict_proba(X_test)[:,fav_idx]

class_thresh = 0.5
dataset_orig_train_pred.scores = y_train_pred_prob.reshape(-1,1)
dataset_orig_valid_pred.scores = y_valid_pred_prob.reshape(-1,1)
dataset_orig_test_pred.scores = y_test_pred_prob.reshape(-1,1)

y_train_pred = np.zeros_like(dataset_orig_train_pred.labels)
y_train_pred[y_train_pred_prob >= class_thresh] = dataset_orig_train_pred.favorable_label
y_train_pred[~(y_train_pred_prob >= class_thresh)] = dataset_orig_train_pred.unfavorable_label
dataset_orig_train_pred.labels = y_train_pred

y_valid_pred = np.zeros_like(dataset_orig_valid_pred.labels)
y_valid_pred[y_valid_pred_prob >= class_thresh] = dataset_orig_valid_pred.favorable_label
y_valid_pred[~(y_valid_pred_prob >= class_thresh)] = dataset_orig_valid_pred.unfavorable_label
dataset_orig_valid_pred.labels = y_valid_pred

y_test_pred = np.zeros_like(dataset_orig_test_pred.labels)
y_test_pred[y_test_pred_prob >= class_thresh] = dataset_orig_test_pred.favorable_label
y_test_pred[~(y_test_pred_prob >= class_thresh)] = dataset_orig_test_pred.unfavorable_label
dataset_orig_test_pred.labels = y_test_pred

In [ ]:
num_thresh = 100
ba_arr = np.zeros(num_thresh)
class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
for idx, class_thresh in enumerate(class_thresh_arr):

    fav_inds = dataset_orig_valid_pred.scores > class_thresh
    dataset_orig_valid_pred.labels[fav_inds] = dataset_orig_valid_pred.favorable_label
    dataset_orig_valid_pred.labels[~fav_inds] = dataset_orig_valid_pred.unfavorable_label

    classified_metric_orig_valid = ClassificationMetric(dataset_orig_valid,
                                             dataset_orig_valid_pred,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

    ba_arr[idx] = 0.5*(classified_metric_orig_valid.true_positive_rate()\
                       +classified_metric_orig_valid.true_negative_rate())

best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
best_class_thresh = class_thresh_arr[best_ind]

print("Best balanced accuracy (no fairness constraints) = %.4f" % np.max(ba_arr))
print("Optimal classification threshold (no fairness constraints) = %.4f" % best_class_thresh)

Best balanced accuracy (no fairness constraints) = 0.7421

Optimal classification threshold (no fairness constraints) = 0.3565